In [1]:
'''
!pip3 install dash
!pip3 install jupyter_dash
!pip3 install numpy
!pip3 install pandas
!pip3 install dash_bootstrap_components
!pip3 install sklearn
!pip3 install osmnx
!pip3 install utm
!pip3 uninstall umap
!pip3 install umap-learn -Y
'''

'''
conda config --prepend channels conda-forge
conda create -n ox --strict-channel-priority osmnx jupyter
conda activate ox
conda install -c conda-forge dash
conda install -c plotly jupyter-dash
conda install -c conda-forge dash-bootstrap-components
conda install -c conda-forge utm
conda install -c conda-forge umap-learn
'''

'\nconda config --prepend channels conda-forge\nconda create -n ox --strict-channel-priority osmnx jupyter\nconda activate ox\nconda install -c conda-forge dash\nconda install -c plotly jupyter-dash\nconda install -c conda-forge dash-bootstrap-components\nconda install -c conda-forge utm\nconda install -c conda-forge umap-learn\n'

In [2]:
import os
import CityHub
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import math
from statistics import mean
from sklearn.manifold import TSNE

In [3]:
# carrega o citygraph preprocessado
ch=CityHub.load_preprocessed_CityHub('data/sp-sibgrapi2022.bin')

In [4]:
def compute_static_feature_vector(vert_ind):

    feature_vec = pd.Series(dtype=float)

    layer_pontodeonibus = 0
    layer_estacaodemetro = 1
    layer_estacaodetrem = 2
    layer_terminaldeonibus = 3

    query_coords=ch.city_vert_list[vert_ind]

#       ibge
    ibge_list = pd.Series([0.0]*7)
    ibge_list.index = ['Renda_media_por_domicilio', 'Renda_media_responsaveis', 'Responsaveis_sem_renda_taxa', 'Alfabetizados_de_7_a_15_anos', 'menores_de_18_anos_taxa', '18_a_65_anos_taxa', 'maiores_de_65_anos_taxa']
    
    for i in range(len(ibge_list.index)):
        feats = ch.polygon_features_from_coords(0, query_coords[0], query_coords[1],  ibge_list.index[i])
        feats = [f for f in feats if not pd.isna(f)]
        if len(feats)==0:
            return pd.Series(dtype=float)
        ibge_list[i]=mean(feats)
    feature_vec = pd.concat([feature_vec, ibge_list])

#       transporte
    transporte_indices = ['Pontos_de_onibus','Estacao_de_metro', 'Estacao_de_trem', 'Terminal_de_onibus']
    transporte_list = pd.Series([0.0]*len(transporte_indices), index = transporte_indices)
    #print(ch.query_point_in_PBLayer(0,query_coords[0],query_coords[1],layer_pontodeonibus,0.2))
    transporte_list['Pontos_de_onibus'] = len(ch.query_point_in_PBLayer(0,query_coords[0],query_coords[1],layer_pontodeonibus,0.2))
    transporte_list['Estacao_de_metro'] = len(ch.query_point_in_PBLayer(1,query_coords[0],query_coords[1],layer_estacaodemetro,0.2))
    transporte_list['Estacao_de_trem'] = len(ch.query_point_in_PBLayer(2,query_coords[0],query_coords[1],layer_estacaodetrem,0.2))
    transporte_list['Terminal_de_onibus'] = len(ch.query_point_in_PBLayer(3,query_coords[0],query_coords[1],layer_terminaldeonibus,0.2))
    #print(transporte_list['Pontos_de_onibus'])
    
    feature_vec = pd.concat([feature_vec, transporte_list])

#       favelas
    favela_indices = ['Favela_proxima']
    favela_list = pd.Series([0.0]*len(favela_indices), index = favela_indices)
    favela_list['Favela_proxima'] = (len(ch.query_point_in_RDLayer(0, query_coords[0],query_coords[1],0.5,True))>0)*1

    feature_vec = pd.concat([feature_vec, favela_list])

    return feature_vec

static_feature_names = ['Renda_media_por_domicilio', 'Renda_media_responsaveis', 'Responsaveis_sem_renda_taxa', 'Alfabetizados_de_7_a_15_anos', 'menores_de_18_anos_taxa', '18_a_65_anos_taxa', 'maiores_de_65_anos_taxa','Pontos_de_onibus','Estacao_de_metro', 'Estacao_de_trem', 'Terminal_de_onibus','Favela_proxima']
#static_feature_names = ['Household_income_avg', 'Householder_income_avg', 'Householder_unemployment_rate', 'Literate_7_15_yrs_children_rate', 'residents_under_18_years_rate', 'residents_aged_18_to_65_years_rate', 'residents_over_65_years_rate','bus_stops','subway_stations', 'train_stations', 'bus_terminals','subnormal_agglomerates_around']

STATIC_FEAT_DIM = len(static_feature_names)

In [5]:
'''
# Teste parar checar que a funcao abaixo esta corretamente computando os dados de clima conforme esperado
# (pode deletar essa célula do notebook para a versão final)
def compute_dynamic_feature_df(vert_ind_series, initial_date = None, final_date = None):
    
    # crimes
    #cg.compute_layer_crimes(initial_date = initial_date, final_date = final_date)
    #feature_df = cg.layer_crimes_count.loc[vert_ind_series]
    
    # weather
    cg.compute_layer_weather(initial_date = initial_date, final_date = final_date)
    feature_df = cg.measu_estimation.loc[vert_ind_series]
    #feature_df = pd.concat([feature_df, cg.measu_estimation.loc[vert_ind_series]], axis = 1)
    
    return feature_df

vert_ind_series = pd.Series(cg.measu_estimation.index).iloc[0::2] # Pandas series com metade de todas as esquinas da cidade
initial_date = pd.to_datetime("01-01-2020",dayfirst=True) # Pandas datetime com data inicial
final_date = pd.to_datetime("31-12-2020",dayfirst=True) # Pandas datetime com data final
compute_dynamic_feature_df(vert_ind_series, initial_date, final_date)
'''

'\n# Teste parar checar que a funcao abaixo esta corretamente computando os dados de clima conforme esperado\n# (pode deletar essa célula do notebook para a versão final)\ndef compute_dynamic_feature_df(vert_ind_series, initial_date = None, final_date = None):\n    \n    # crimes\n    #cg.compute_layer_crimes(initial_date = initial_date, final_date = final_date)\n    #feature_df = cg.layer_crimes_count.loc[vert_ind_series]\n    \n    # weather\n    cg.compute_layer_weather(initial_date = initial_date, final_date = final_date)\n    feature_df = cg.measu_estimation.loc[vert_ind_series]\n    #feature_df = pd.concat([feature_df, cg.measu_estimation.loc[vert_ind_series]], axis = 1)\n    \n    return feature_df\n\nvert_ind_series = pd.Series(cg.measu_estimation.index).iloc[0::2] # Pandas series com metade de todas as esquinas da cidade\ninitial_date = pd.to_datetime("01-01-2020",dayfirst=True) # Pandas datetime com data inicial\nfinal_date = pd.to_datetime("31-12-2020",dayfirst=True) # Panda

In [6]:
import types
def compute_layer_crimes(self, layer, initial_date = None, final_date = None):

        tmp_layer_crimes = self.PBLayers[layer].copy()
        if((not initial_date is None) or (not final_date is None)):
            tmp_layer_crimes["DATAOCORRENCIA"] = pd.to_datetime(tmp_layer_crimes["DATAOCORRENCIA"], format = "%d/%m/%Y", errors = 'coerce')
            tmp_layer_crimes = tmp_layer_crimes[tmp_layer_crimes['DATAOCORRENCIA'].notna()]
            if(not initial_date is None):
                tmp_layer_crimes = tmp_layer_crimes[tmp_layer_crimes["DATAOCORRENCIA"] >= initial_date]
            if(not final_date is None):
                tmp_layer_crimes = tmp_layer_crimes[tmp_layer_crimes["DATAOCORRENCIA"] <= final_date]

        aux = pd.DataFrame(tmp_layer_crimes[["vert","OBJETO"]].value_counts().keys().tolist(), columns = ["vert","OBJETO"])

        aux["n"] = tmp_layer_crimes[["vert","OBJETO"]].value_counts().values
        aux = aux.pivot(index='vert', columns='OBJETO')
        aux.columns = [ x[1] for x in aux.columns ]

        aux["TODOS"] = aux.sum(axis=1)

        crimes_columns = list(tmp_layer_crimes['OBJETO'].unique())
        crimes_count = [[0]*(len(crimes_columns)+1) for _ in range(len(self.city_vert_dict.values()))]

        crimes_count = pd.DataFrame(crimes_count,columns = aux.columns) + aux
        crimes_count = crimes_count.fillna(0)
        
           
        crimes_count.columns = ['CRIME_' + name.replace(' ','_') for name in list(crimes_count.columns)]

        #self.layer_crimes_vertices = vertices_points
        self.layer_crimes_count = crimes_count
        self.layer_crimes_count.rename(columns={"CRIME_CELULAR": "crime_mobile", "CRIME_VEÍCULOS": "crime_vehicle", "CRIME_TODOS": "crime_all"},inplace=True)

        return True
    
ch.compute_layer_crimes = types.MethodType(compute_layer_crimes, ch)
ch.compute_layer_crimes(4)

True

In [7]:
def compute_dynamic_feature_df(vert_ind_series, initial_date = None, final_date = None):
    
    # crimes
    ch.compute_layer_crimes(layer = 4, initial_date = initial_date, final_date = final_date)
    feature_df = ch.layer_crimes_count.loc[vert_ind_series]
    
    # weather
    ch.compute_layer_sparse(initial_date, final_date)
    #ch.compute_layer_weather(initial_date = initial_date, final_date = final_date)
    feature_df = pd.concat([feature_df, ch.SMLayers_estimated_measurements.loc[vert_ind_series]], axis = 1)
    
    feature_df.rename(columns={'Precipitacao_Total': 'precipitation_total', 'Temperatura_Maxima': 'temperature_max', 'Temperatura_Minima': 'temperature_min'}, inplace=True)
    
    return feature_df

dynamic_feature_names = list(ch.layer_crimes_count.columns) + ['precipitation_total','temperature_max','temperature_min']
DYNAMIC_FEAT_DIM = len(dynamic_feature_names)

In [8]:
dynamic_feature_names

['crime_mobile',
 'crime_vehicle',
 'crime_all',
 'precipitation_total',
 'temperature_max',
 'temperature_min']

In [9]:
#define a função extratora de features
'''
def compute_feature_vector(vert_ind):

    feature_vec = pd.Series(dtype=float)

    layer_pontodeonibus = 0
    layer_estacaodemetro = 1
    layer_estacaodetrem = 2
    layer_terminaldeonibus = 3

    query_coords=cg.city_vert_list[vert_ind]

#       crimes

    feature_vec = pd.concat([feature_vec, pd.Series(cg.layer_crimes_count.loc[vert_ind])])

#       ibge
    ibge_list = pd.Series([0.0]*7)
    ibge_list.index = ['Renda_media_por_domicilio', 'Renda_media_responsaveis', 'Responsaveis_sem_renda_taxa', 'Alfabetizados_de_7_a_15_anos', 'menores_de_18_anos_taxa', '18_a_65_anos_taxa', 'maiores_de_65_anos_taxa']

    for i in range(len(ibge_list.index)):
        feats = cg.polygon_features_from_vertex(vert_ind,  ibge_list.index[i])
        if len(feats)==0:
            return pd.Series(dtype=float)
        for f in feats:
            if math.isnan(f):
                return pd.Series(dtype=float)
        ibge_list[i]=mean(feats)
    feature_vec = pd.concat([feature_vec, ibge_list])

#       transporte
    transporte_indices = ['Pontos_de_onibus','Estacao_de_metro', 'Estacao_de_trem', 'Terminal_de_onibus']
    transporte_list = pd.Series([0.0]*len(transporte_indices), index = transporte_indices)
    transporte_list['Pontos_de_onibus'] = len(cg.query_point_in_points_layer(query_coords[0],query_coords[1],layer_pontodeonibus,0.2))
    transporte_list['Estacao_de_metro'] = len(cg.query_point_in_points_layer(query_coords[0],query_coords[1],layer_estacaodemetro,0.2))
    transporte_list['Estacao_de_trem'] = len(cg.query_point_in_points_layer(query_coords[0],query_coords[1],layer_estacaodetrem,0.2))
    transporte_list['Terminal_de_onibus'] = len(cg.query_point_in_points_layer(query_coords[0],query_coords[1],layer_terminaldeonibus,0.2))

    feature_vec = pd.concat([feature_vec, transporte_list])

#       favelas
    favela_indices = ['Favela_proxima']
    favela_list = pd.Series([0.0]*len(favela_indices), index = favela_indices)
    favela_list['Favela_proxima'] = (len(cg.query_point_in_mesh_layer(query_coords[0],query_coords[1],0,0.5,True))>0)*1

    feature_vec = pd.concat([feature_vec, favela_list])
    
#       weather
    feature_vec = feature_vec.append(cg.measu_estimation.loc[vert_ind,:])

    return feature_vec
'''

"\ndef compute_feature_vector(vert_ind):\n\n    feature_vec = pd.Series(dtype=float)\n\n    layer_pontodeonibus = 0\n    layer_estacaodemetro = 1\n    layer_estacaodetrem = 2\n    layer_terminaldeonibus = 3\n\n    query_coords=cg.city_vert_list[vert_ind]\n\n#       crimes\n\n    feature_vec = pd.concat([feature_vec, pd.Series(cg.layer_crimes_count.loc[vert_ind])])\n\n#       ibge\n    ibge_list = pd.Series([0.0]*7)\n    ibge_list.index = ['Renda_media_por_domicilio', 'Renda_media_responsaveis', 'Responsaveis_sem_renda_taxa', 'Alfabetizados_de_7_a_15_anos', 'menores_de_18_anos_taxa', '18_a_65_anos_taxa', 'maiores_de_65_anos_taxa']\n\n    for i in range(len(ibge_list.index)):\n        feats = cg.polygon_features_from_vertex(vert_ind,  ibge_list.index[i])\n        if len(feats)==0:\n            return pd.Series(dtype=float)\n        for f in feats:\n            if math.isnan(f):\n                return pd.Series(dtype=float)\n        ibge_list[i]=mean(feats)\n    feature_vec = pd.concat([

In [10]:
FEAT_DIM = len(static_feature_names + dynamic_feature_names)

In [11]:
#filtra a bbox espacial de esquinas, calcula os features de todas as esquinas da bbox e salva a matriz X e a indexação de esquinas nos arquivos X.csv e tabela.csv
#esse bloco é lento e sua execução pode ser realizada apenas uma vez (posteriormente apenas carrega-se os arquivos csv)
'''
bbox = [-23.605011547505715, -46.75149838501082, -23.505550348734968, -46.583734867389815]

num_esquinas_na_bbox = 0
for i in range(len(cg.city_vert_list)):
    latlong=cg.city_vert_list[i]
    if((bbox[0] <= latlong[0] <= bbox[2]) and (bbox[1] <= latlong[1] <= bbox[3])):
     #   print(i)
        num_esquinas_na_bbox += 1


X = np.zeros((num_esquinas_na_bbox,FEAT_DIM))
tabela_esquinas_bbox = [0] * num_esquinas_na_bbox
proxima=0

for i in range(len(cg.city_vert_list)):
    latlong=cg.city_vert_list[i]
    if((bbox[0] <= latlong[0] <= bbox[2]) and (bbox[1] <= latlong[1] <= bbox[3])):
        f = compute_feature_vector(i).to_numpy()
        if(len(f)<FEAT_DIM):
            continue
        tabela_esquinas_bbox[proxima] = i
        X[proxima,:]=f
        proxima += 1
        if ( proxima % 100 == 0):
            print(proxima)

X=X[:proxima, :]
tabela_esquinas_bbox = tabela_esquinas_bbox[:proxima]

df = pd.DataFrame(X)
df.to_csv('X.csv')

import csv
with open('tabela.csv', 'w', newline='') as csvfile:
     wr = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
     wr.writerow(tabela_esquinas_bbox)
'''

"\nbbox = [-23.605011547505715, -46.75149838501082, -23.505550348734968, -46.583734867389815]\n\nnum_esquinas_na_bbox = 0\nfor i in range(len(cg.city_vert_list)):\n    latlong=cg.city_vert_list[i]\n    if((bbox[0] <= latlong[0] <= bbox[2]) and (bbox[1] <= latlong[1] <= bbox[3])):\n     #   print(i)\n        num_esquinas_na_bbox += 1\n\n\nX = np.zeros((num_esquinas_na_bbox,FEAT_DIM))\ntabela_esquinas_bbox = [0] * num_esquinas_na_bbox\nproxima=0\n\nfor i in range(len(cg.city_vert_list)):\n    latlong=cg.city_vert_list[i]\n    if((bbox[0] <= latlong[0] <= bbox[2]) and (bbox[1] <= latlong[1] <= bbox[3])):\n        f = compute_feature_vector(i).to_numpy()\n        if(len(f)<FEAT_DIM):\n            continue\n        tabela_esquinas_bbox[proxima] = i\n        X[proxima,:]=f\n        proxima += 1\n        if ( proxima % 100 == 0):\n            print(proxima)\n\nX=X[:proxima, :]\ntabela_esquinas_bbox = tabela_esquinas_bbox[:proxima]\n\ndf = pd.DataFrame(X)\ndf.to_csv('X.csv')\n\nimport csv\nwit

In [12]:
# Encontrar os vertices das esquinas que caem dentro do bbox usando geopandas e sjoin

from shapely.geometry import Polygon, Point

bbox = gpd.GeoDataFrame(geometry = [Polygon([(-23.605011547505715, -46.583734867389815), (-23.505550348734968, -46.583734867389815), (-23.505550348734968, -46.75149838501082), (-23.605011547505715, -46.75149838501082)])])

city_vert_gdf = gpd.GeoDataFrame(list(range(0,len(ch.city_vert_list))),geometry = gpd.points_from_xy(pd.DataFrame(ch.city_vert_list)[0], pd.DataFrame(ch.city_vert_list)[1]))
city_vert_gdf.columns = ["vert", "geometry"]

tabela_gpd = gpd.sjoin(city_vert_gdf, bbox)[["vert","geometry"]]

In [13]:
tabela_gpd["vert"].to_csv("tabela_gpd.csv")

In [14]:
#carrega a tabela de indexação de esquinas dos arquivos csv para comparar com o novo método mais rápido
'''
import csv
with open('tabela.csv', newline='') as csvfile:
     tab = list(csv.reader(csvfile))[0]
tabela_esquinas_bbox = [int(item) for item in tab] 
'''

"\nimport csv\nwith open('tabela.csv', newline='') as csvfile:\n     tab = list(csv.reader(csvfile))[0]\ntabela_esquinas_bbox = [int(item) for item in tab] \n"

In [15]:
# Verificar quantas esquinas tem em tabela_esquinas_bbox que não tem em tabela_gpd -> resposta nenhuma
# Todas as esquinas que tem em tabela_esquinas_bbox tem em tabela_gpd
#len(set(tabela_esquinas_bbox) - set(tabela_gpd["vert"]))

In [16]:
# Porém temos 1305 esquinas a mais em tabela_gpd que em tabela_esquinas_bbox
# Razão - esses 1305 são pontos em que não foi possível calcular os dados do IBGE no método anterior
#len(set(tabela_gpd["vert"]) - set(tabela_esquinas_bbox))

In [17]:
#Para verificar se os vertices de tabela_gpd estão caindo dentro do bbox usando gpd -> resposta sim
'''
import folium
import osmnx as ox
import networkx as nx

lat = -23.549090314066356
long = -46.65458954194309

rectangle_center = [lat, long]

m = folium.Map(location=[lat, long], zoom_start=15, tiles='CartoDB positron')

bbox_aux = gpd.GeoDataFrame(geometry = [Polygon([(-46.583734867389815,-23.605011547505715), (-46.583734867389815, -23.505550348734968), (-46.75149838501082, -23.505550348734968), (-46.75149838501082, -23.605011547505715)])])
sim_geo = gpd.GeoSeries(bbox_aux.iloc[0]['geometry']).simplify(tolerance=0.001)
geo_j = sim_geo.to_json()
geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': 'orange'})
folium.Popup(["Geopandas"]).add_to(geo_j)
geo_j.add_to(m)

for v in tabela_gpd["vert"]:
    folium.CircleMarker(location=[cg.city_vert_list[v][0], cg.city_vert_list[v][1]], radius=1, color = "red").add_to(m)

m
'''

'\nimport folium\nimport osmnx as ox\nimport networkx as nx\n\nlat = -23.549090314066356\nlong = -46.65458954194309\n\nrectangle_center = [lat, long]\n\nm = folium.Map(location=[lat, long], zoom_start=15, tiles=\'CartoDB positron\')\n\nbbox_aux = gpd.GeoDataFrame(geometry = [Polygon([(-46.583734867389815,-23.605011547505715), (-46.583734867389815, -23.505550348734968), (-46.75149838501082, -23.505550348734968), (-46.75149838501082, -23.605011547505715)])])\nsim_geo = gpd.GeoSeries(bbox_aux.iloc[0][\'geometry\']).simplify(tolerance=0.001)\ngeo_j = sim_geo.to_json()\ngeo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {\'fillColor\': \'orange\'})\nfolium.Popup(["Geopandas"]).add_to(geo_j)\ngeo_j.add_to(m)\n\nfor v in tabela_gpd["vert"]:\n    folium.CircleMarker(location=[cg.city_vert_list[v][0], cg.city_vert_list[v][1]], radius=1, color = "red").add_to(m)\n\nm\n'

In [18]:
%%time
#Calcular X das features dinâmicas
X_dynamic = compute_dynamic_feature_df(tabela_gpd["vert"],'2020-01-01','2020-12-31')

Wall time: 1.85 s


In [19]:
X_dynamic

,crime_mobile,crime_vehicle,crime_all,precipitation_total,temperature_max,temperature_min
0,14.0,12.0,26.0,1372.100297,37.557652,7.558559
1,4.0,0.0,4.0,1372.519998,37.565831,7.568262
2,0.0,0.0,0.0,1374.435427,37.576437,7.583988
3,1.0,0.0,1.0,1369.868512,37.573657,7.570547
4,2.0,0.0,2.0,1288.175711,37.502308,7.307016
...,...,...,...,...,...,...
119682,0.0,0.0,0.0,1357.506256,37.596945,7.567097
119714,22.0,0.0,22.0,1530.703484,37.522692,7.884789
119715,30.0,1.0,31.0,1531.738406,37.525682,7.890357
119723,0.0,1.0,1.0,1584.343383,37.484901,7.967368


In [20]:
%%time
#Calcular X das features estáticas
X_static = tabela_gpd["vert"].apply(lambda vert: compute_static_feature_vector(vert))

Wall time: 42min 4s


In [21]:
X_static = X_static.dropna()
#X_static.to_csv("X_static.csv")

In [22]:
#carrega o array x de features estáticas e a tabela de indexação de esquinas dos arquivos csv

#X_static = pd.read_csv("X_static.csv",index_col = 0)
tabela_esquinas_bbox = list(X_static.index)

In [23]:
X_static

,Renda_media_por_domicilio,Renda_media_responsaveis,Responsaveis_sem_renda_taxa,Alfabetizados_de_7_a_15_anos,menores_de_18_anos_taxa,18_a_65_anos_taxa,maiores_de_65_anos_taxa,Pontos_de_onibus,Estacao_de_metro,Estacao_de_trem,Terminal_de_onibus,Favela_proxima
0,17703.084229,10920.004934,0.066325,0.991071,0.210939,0.631893,0.157168,0.0,1.0,3.0,1.0,0.0
1,3331.532558,2068.113256,0.119709,0.906177,0.169365,0.648065,0.182571,0.0,3.0,3.0,1.0,0.0
2,12941.995652,9115.711180,0.077019,1.000000,0.125065,0.742215,0.132719,0.0,3.0,3.0,1.0,0.0
3,12695.832272,9088.416954,0.100086,1.000000,0.117566,0.794885,0.087549,0.0,2.0,4.0,1.0,0.0
4,3720.941215,1987.940523,0.141944,1.000000,0.190903,0.668093,0.141004,0.0,0.0,4.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
119682,25745.704040,19362.012121,0.140909,1.000000,0.170204,0.673921,0.155875,0.0,0.0,3.0,1.0,0.0
119714,2143.704684,1106.493097,0.119356,0.951604,0.255950,0.684830,0.059219,0.0,0.0,5.0,6.0,0.0
119715,2485.378454,1295.751551,0.042129,0.951215,0.251850,0.697057,0.051093,0.0,1.0,5.0,5.0,0.0
119723,3150.869074,1615.765975,0.109932,0.979724,0.184311,0.659699,0.155991,0.0,0.0,0.0,0.0,0.0


In [24]:
X_static.columns = ['Household_income_avg', 'Householder_income_avg', 'Householder_unemployment_rate', 'Literate_7_15_yrs_children_rate', 'residents_under_18_years_rate', 'residents_aged_18_to_65_years_rate', 'residents_over_65_years_rate','bus_stops','subway_stations', 'train_stations', 'bus_terminals','subnormal_agglomerates_around']

In [25]:
X_static

,Household_income_avg,Householder_income_avg,Householder_unemployment_rate,Literate_7_15_yrs_children_rate,residents_under_18_years_rate,residents_aged_18_to_65_years_rate,residents_over_65_years_rate,bus_stops,subway_stations,train_stations,bus_terminals,subnormal_agglomerates_around
0,17703.084229,10920.004934,0.066325,0.991071,0.210939,0.631893,0.157168,0.0,1.0,3.0,1.0,0.0
1,3331.532558,2068.113256,0.119709,0.906177,0.169365,0.648065,0.182571,0.0,3.0,3.0,1.0,0.0
2,12941.995652,9115.711180,0.077019,1.000000,0.125065,0.742215,0.132719,0.0,3.0,3.0,1.0,0.0
3,12695.832272,9088.416954,0.100086,1.000000,0.117566,0.794885,0.087549,0.0,2.0,4.0,1.0,0.0
4,3720.941215,1987.940523,0.141944,1.000000,0.190903,0.668093,0.141004,0.0,0.0,4.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
119682,25745.704040,19362.012121,0.140909,1.000000,0.170204,0.673921,0.155875,0.0,0.0,3.0,1.0,0.0
119714,2143.704684,1106.493097,0.119356,0.951604,0.255950,0.684830,0.059219,0.0,0.0,5.0,6.0,0.0
119715,2485.378454,1295.751551,0.042129,0.951215,0.251850,0.697057,0.051093,0.0,1.0,5.0,5.0,0.0
119723,3150.869074,1615.765975,0.109932,0.979724,0.184311,0.659699,0.155991,0.0,0.0,0.0,0.0,0.0


In [26]:
# gerando df para com features estáticas e dinâmicas
# é possível incluir initial_date e final_date em compute_dynamic_feature_df
df = pd.concat([X_static, compute_dynamic_feature_df(tabela_esquinas_bbox,'2020-01-01','2020-12-31')], axis = 1)

In [27]:
import umap.umap_ as umap
reducer = umap.UMAP()
df_normalized=(df-df.mean())/df.std()
df_normalized=df_normalized.fillna(0)
X_embedded = reducer.fit_transform(df_normalized)

In [28]:
df['lat'] =[ch.city_vert_list[i][0] for i in tabela_esquinas_bbox]
df['lon'] = [ch.city_vert_list[i][1] for i in tabela_esquinas_bbox]
df['u'] = X_embedded[:,0]
df['v'] = X_embedded[:,1]

In [29]:
def compute_histograms(df_name, nbins = 18):
    hists = dict()
    bins = dict()
    for col in df_name.columns[:-4]:
        hists[col],bins[col]=np.histogram(df_name[col], bins=nbins, density=True)
    return hists,bins

In [30]:
full_hists,full_bins = compute_histograms(df)

In [31]:
def compute_histograms_definedbins(df_name, definedbins):
    hists = dict()
    bins = dict()
    for col in df_name.columns[:-4]:
        hists[col],bins[col]=np.histogram(df_name[col], bins=definedbins[col], density=True)
    return hists,bins

In [32]:
partial_hists,partial_bins = compute_histograms_definedbins(df,full_bins)

In [33]:
# calculate the kl divergence
#def kl_divergence(p, q):
#   return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))
from scipy.stats import wasserstein_distance

def kl(p, q):
    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)

    return np.sum(np.where(p != 0, p * np.log(p / q), 0.0))


def hist_euc(p, q):
    return np.linalg.norm(p/np.linalg.norm(p)-q/np.linalg.norm(q))


def dict_divergence(dict1,dict2):
    dict_div = dict()
    for k, v in dict1.items():
        dict_div[k]=hist_euc(v,dict2[k])    
    #   dict_div[k]=wasserstein_distance(v,dict2[k])
    #    if k=='Crimes':
    #    print(dict_div[k])
    return dict_div

In [34]:
df_backup=df
df_backup

,Household_income_avg,Householder_income_avg,Householder_unemployment_rate,Literate_7_15_yrs_children_rate,residents_under_18_years_rate,residents_aged_18_to_65_years_rate,residents_over_65_years_rate,bus_stops,subway_stations,train_stations,...,crime_mobile,crime_vehicle,crime_all,precipitation_total,temperature_max,temperature_min,lat,lon,u,v
0,17703.084229,10920.004934,0.066325,0.991071,0.210939,0.631893,0.157168,0.0,1.0,3.0,...,14.0,12.0,26.0,1372.100297,37.557652,7.558559,-23.562991,-46.703965,5.602483,4.673079
1,3331.532558,2068.113256,0.119709,0.906177,0.169365,0.648065,0.182571,0.0,3.0,3.0,...,4.0,0.0,4.0,1372.519998,37.565831,7.568262,-23.567465,-46.702229,-0.292905,0.517573
2,12941.995652,9115.711180,0.077019,1.000000,0.125065,0.742215,0.132719,0.0,3.0,3.0,...,0.0,0.0,0.0,1374.435427,37.576437,7.583988,-23.572962,-46.699235,7.145585,4.222012
3,12695.832272,9088.416954,0.100086,1.000000,0.117566,0.794885,0.087549,0.0,2.0,4.0,...,1.0,0.0,1.0,1369.868512,37.573657,7.570547,-23.571831,-46.702075,0.483070,0.400319
4,3720.941215,1987.940523,0.141944,1.000000,0.190903,0.668093,0.141004,0.0,0.0,4.0,...,2.0,0.0,2.0,1288.175711,37.502308,7.307016,-23.534616,-46.748148,16.301880,2.297124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119682,25745.704040,19362.012121,0.140909,1.000000,0.170204,0.673921,0.155875,0.0,0.0,3.0,...,0.0,0.0,0.0,1357.506256,37.596945,7.567097,-23.584573,-46.704136,8.669142,4.736488
119714,2143.704684,1106.493097,0.119356,0.951604,0.255950,0.684830,0.059219,0.0,0.0,5.0,...,22.0,0.0,22.0,1530.703484,37.522692,7.884789,-23.538866,-46.620479,2.339030,13.517008
119715,2485.378454,1295.751551,0.042129,0.951215,0.251850,0.697057,0.051093,0.0,1.0,5.0,...,30.0,1.0,31.0,1531.738406,37.525682,7.890357,-23.541062,-46.617960,1.677951,13.115665
119723,3150.869074,1615.765975,0.109932,0.979724,0.184311,0.659699,0.155991,0.0,0.0,0.0,...,0.0,1.0,1.0,1584.343383,37.484901,7.967368,-23.511860,-46.594873,6.936034,14.189716


In [35]:
import dash
from dash import dcc, ctx
from dash import html
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import numpy as np
import pandas as pddict_divergence
import plotly.express as px
import plotly.graph_objs as go
import dash_bootstrap_components as dbc
from datetime import date

#app = JupyterDash()
selected_corners_indices = []
app = JupyterDash(external_stylesheets=[dbc.themes.PULSE])
server = app.server
app.index_string = """
<!DOCTYPE html>
<html>
   <head>
       {%metas%}
       <title>{%title%}</title>
       {%favicon%}
       {%css%}
       <style>

            .DateInput_input__focused{
                border-bottom: 2px solid #0175ff;
            }
            .DateInput_input, .DateInput_input_1 {
                font-family: Arial;
                font-size: 10px; 
                height: 10px;
            }
            .DateInput, .DateInput_1 {
                width: 40%;
                height: inherit;
            }
       </style>
   </head>
   <body>
       {%app_entry%}
       <footer>
           {%config%}
           {%scripts%}
           {%renderer%}
       </footer>
   </body>
</html>
"""
df=df_backup.copy()
fig = px.scatter_mapbox(df, lat="lat", lon="lon", hover_name = df.index, hover_data=df.columns[:-2],
                        color_discrete_sequence=['#00a699'], zoom=3, height=300)#"fuchsia"
fig.update_layout(mapbox_style="open-street-map",
                  margin={"r":0,"t":0,"l":0,"b":0},
                  autosize=True,
                  mapbox = {
                'center': {'lon': -46.625290, 'lat': -23.533773},
                'zoom': 10},
                  height=450
                 )

scatter = go.Figure()
scatter.add_trace(go.Scattergl(
                x = df.u,
                y = df.v,
                mode = 'markers',
                marker = dict(
                color='#00a699', #green 
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
))
scatter.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    dragmode='select',
    clickmode='event+select')

dict_div=dict_divergence(full_hists,full_hists)
data = [go.Bar(x=list(dict_div.keys()),y=list(dict_div.values()))]
fig_barplot = go.Figure(data=data)
fig_barplot.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
fig_barplot.update_xaxes(tickfont_size=10)

def blank_fig(text):
    fig = go.Figure(go.Scatter(x=[], y = []))
    fig.update_layout(
          title_text=text,title_font_size=10,
        template = None,height=30)
    fig.update_xaxes(showgrid = False, showticklabels = False, zeroline=False)
    fig.update_yaxes(showgrid = False, showticklabels = False, zeroline=False)
    fig.update_layout({"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"})
    return fig

app.layout = dbc.Container(
    [
        
        html.H1(),
        html.Center(html.H4(children='CityHub Visual Explorer')),
        
        html.Hr(),
         dbc.Row(
                [
                    dbc.Col([
                        dbc.Row(html.H5(children='Features')),
                        dbc.Row(html.H6(children='Static')), 
                        dbc.Row(          
                           dcc.Checklist(
                               id="features_checklist_static",
                               options=static_feature_names,
                               value=static_feature_names,
                               style={'font-size': '10px'},
                               labelStyle={'display': 'block',} ),                
                        ),
                        dbc.Row(html.H6(children='Dynamic')), 
                        dbc.Row(
                            dcc.DatePickerRange(
                                id='my-date-picker-range',
                                start_date=date(2020, 1, 1),
                                end_date=date(2020, 12, 31),
                                min_date_allowed = date(2020, 1, 1),
                                max_date_allowed = date(2020, 12, 31),
                            )
                        ),
                        html.H2(),
                        dbc.Row(          
                           dcc.Checklist(
                               id="features_checklist_dynamic",
                               options=dynamic_feature_names,
                               value=dynamic_feature_names,
                               style={'font-size': '10px'},
                               labelStyle={'display': 'block',} ),                
                        ),
                        html.H2(),
                        dbc.Button(
                            str('Update'),
                            className="btn btn-primary",
                            id='update_button',
                            n_clicks=0,
                        )
                        ], width=2),
                    dbc.Col([
                        dbc.Row(html.H5(children='Projection View')),                     
                        dbc.Row(dcc.Graph(id="projectionScatter", figure=scatter)),
                            ], width=5),
                    dbc.Col([
                        dbc.Row(html.H5(children='Map View')),
                        dbc.Row(dcc.Graph(id="mymap", figure=fig))
                        ], width=5),
                ]
            ),
        dbc.Row(
                [
                    dbc.Col([
                        dbc.Row(html.H5(children='Features distribution')),  
                        dbc.Row(html.H6(children='Global')), 
                        dbc.Row(dcc.Graph(id="general",figure= blank_fig(''))),
                        dbc.Row(html.H6(children='Selected')),  
                        dbc.Row(dcc.Graph(id="detail",figure= blank_fig(''))),
                            ], width=2),
                    dbc.Col([
                        dbc.Row(html.H5(children='Feature Histogram Distance')),                     
                        dbc.Row(dcc.Graph(id="barplot", figure=fig_barplot)),
                            ], width=10),
                ]
            ),
        dcc.Store(id="features-detail-df"),
        dcc.Store(id="features-general-df"),
    ]
)




def scatterplot(x, y, title="", axis_type="Linear"):
    return {
        "data": [go.Scatter(x=df.u, y=df.v, mode="markers") ],
    }

@app.callback(
    Output("barplot", "figure"),
    Output("projectionScatter", "figure"),
    Output("mymap", "figure"),
    Output('features-detail-df', 'data'),
    Output('features-general-df', 'data'),
    [
        Input("features_checklist_static", "value"),
        Input("features_checklist_dynamic", "value"),
        Input('my-date-picker-range', 'start_date'),
        Input('my-date-picker-range', 'end_date'),
        Input("projectionScatter", "selectedData"),
        Input("projectionScatter", "figure"),
        Input("barplot", "figure"),
        Input("mymap", "selectedData"),
        Input("mymap", "figure"),
        Input('features-detail-df', 'data'),
        Input('features-general-df', 'data'),
        Input('update_button', 'n_clicks'),
    ], prevent_initial_call=True
)
def update_cb(options_chosen_static, options_chosen_dynamic, start_date, end_date, selectedData, scatter_current_fig, barplot_current_fig, selectedDataMap, map_current_fig,features_current_data,features_current_data_general,n_clicks):
    triggered_id = ctx.triggered_id
    global selected_corners_indices
    if triggered_id == 'mymap':
        if len(selectedDataMap["points"])==0:
            return barplot_current_fig,scatter_current_fig,map_current_fig,features_current_data,features_current_data_general
        df2=pd.concat([X_static, compute_dynamic_feature_df(tabela_esquinas_bbox,start_date,end_date),df.lat,df.lon,df.u,df.v], axis = 1)
        temp=df2
        pt = selectedDataMap["points"]
        #indices = []
        selected_corners_indices.clear()
        for sel in pt:
            #indices.append(sel['hovertext'])
            selected_corners_indices.append(sel['hovertext'])
        selected_corners_indices = list(set(selected_corners_indices))
        #temp=temp.loc[indices]
        temp=temp.iloc[selected_corners_indices]
        full_hists,full_bins = compute_histograms(df2)
        partial_hists,partial_bins = compute_histograms_definedbins(temp,full_bins)
        dict_div=dict_divergence(full_hists,partial_hists)
        data = [go.Bar(x=list(dict_div.keys()),y=list(dict_div.values()))]
        fig_barplot = go.Figure(data=data)
        fig_barplot.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
        fig_barplot.update_xaxes(tickfont_size=10)
        scatter = go.Figure()
        scatter.add_trace(go.Scattergl(
                x = df.u,#scatter_current_fig['data'][0]['x'],
                y = df.v,#scatter_current_fig['data'][0]['y'],
                mode = 'markers',
                marker = dict(
                color='#00a699',#'green',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.add_trace(go.Scattergl(
                x = temp.u,
                y = temp.v,
                mode = 'markers',
                marker = dict(
                #color='#5a3685',#'blue',
                color = 'red',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.update_layout(clickmode='event+select',
                              dragmode='select',
                              margin ={'l':0,'t':0,'b':0,'r':0},
                              showlegend=False) 
        return fig_barplot,scatter,map_current_fig,temp.to_json(date_format='iso', orient='split'), df2.to_json(date_format='iso', orient='split')

    elif triggered_id == 'projectionScatter':
        df2=pd.concat([X_static, compute_dynamic_feature_df(tabela_esquinas_bbox,start_date,end_date),df.lat,df.lon,df.u,df.v], axis = 1)
        #options_chosen = options_chosen_static + options_chosen_dynamic
        temp = df2
        temp["lat"] = df["lat"]
        temp["lon"] = df["lon"]
        pt = pd.DataFrame(selectedData["points"])
        pt = pt[pt.curveNumber == 0]
        #indices = []
        selected_corners_indices.clear()
        for sel in pt['pointIndex']:
            #indices.append(sel['pointIndex'])
            selected_corners_indices.append(sel)
        #temp=temp.iloc[indices]
        selected_corners_indices = list(set(selected_corners_indices))
        temp=temp.iloc[selected_corners_indices]
        mapfig = px.scatter_mapbox(temp, lat="lat", lon="lon", hover_name=temp.index, hover_data=temp.columns[:-2],
                        color_discrete_sequence=['#636efa'], zoom=3, height=300)#'blue'
        mapfig.update_layout(mapbox_style="open-street-map",
                  margin={"r":0,"t":0,"l":0,"b":0},
                  autosize=True,
                  mapbox = {
                'center': {'lon': -46.625290, 'lat': -23.533773},
                'zoom': 10},
                  height=450
                 )
        scatter = go.Figure()
        scatter.add_trace(go.Scattergl(
                x = df2.u,#scatter_current_fig['data'][0]['x'],
                y = df2.v,#scatter_current_fig['data'][0]['y'],
                mode = 'markers',
                marker = dict(
                color='#00a699',#'green',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.add_trace(go.Scattergl(
                x = temp.u,
                y = temp.v,
                mode = 'markers',
                marker = dict(
                #color='#5a3685',#'blue',
                color = 'red',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.update_layout(clickmode='event+select',
                              dragmode='select',
                              margin ={'l':0,'t':0,'b':0,'r':0},
                              showlegend=False) 
        full_hists,full_bins = compute_histograms(df2)
        partial_hists,partial_bins = compute_histograms_definedbins(temp,full_bins)
        dict_div=dict_divergence(full_hists,partial_hists)
        data = [go.Bar(x=list(dict_div.keys()),y=list(dict_div.values()))]
        fig_barplot = go.Figure(data=data)
        fig_barplot.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
        fig_barplot.update_xaxes(tickfont_size=10)
        return fig_barplot,scatter,mapfig,temp.to_json(date_format='iso', orient='split'),df2.to_json(date_format='iso', orient='split')
    
    elif triggered_id == 'update_button': #or triggered_id =='features_checklist_static' or triggered_id == 'features_checklist_dynamic' or triggered_id == 'my-date-picker-range':
        df2=pd.concat([X_static, compute_dynamic_feature_df(tabela_esquinas_bbox,start_date,end_date),df.lat,df.lon,df.u,df.v], axis = 1)
        print(df2.columns)
        options_chosen = options_chosen_static + options_chosen_dynamic
        temp = df2.filter(options_chosen)
        temp_normalized=(temp-temp.mean())/temp.std()
        temp_normalized.fillna(0)
        X_embedded = reducer.fit_transform(temp_normalized)
        temp['u'] = X_embedded[:,0]
        temp['v'] = X_embedded[:,1]
        df.u=temp.u
        df.v=temp.v
        df2.u=temp.u
        df2.v=temp.v
        selected_df = df2.iloc[selected_corners_indices]
        scatter = go.Figure()
        scatter.add_trace(go.Scattergl(
                x = temp.u,
                y = temp.v,
                mode = 'markers',
                marker = dict(
                color='#00a699',#'green',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.add_trace(go.Scattergl(
                x = selected_df.u,
                y = selected_df.v,
                mode = 'markers',
                marker = dict(
                #color='#5a3685',#'blue',
                color = 'red',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.update_layout(clickmode='event+select',
                              dragmode='select',
                              margin ={'l':0,'t':0,'b':0,'r':0},
                              showlegend = False)       
        mapfig = px.scatter_mapbox(selected_df, lat="lat", lon="lon", hover_name=selected_df.index,  hover_data=selected_df.columns[:-2],
                        color_discrete_sequence=['#636efa'], zoom=3, height=300)#"blue"
        mapfig.update_layout(mapbox_style="open-street-map",
                  margin={"r":0,"t":0,"l":0,"b":0},
                  autosize=True,
                  mapbox = {
                'center': {'lon': -46.625290, 'lat': -23.533773},
                'zoom': 10},
                  height=450,
                 )  
        full_hists,full_bins = compute_histograms(df2)
        partial_hists,partial_bins = compute_histograms_definedbins(selected_df,full_bins)
        dict_div=dict_divergence(full_hists,partial_hists)
        data = [go.Bar(x=list(dict_div.keys()),y=list(dict_div.values()))]
        fig_barplot = go.Figure(data=data)
        fig_barplot.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
        fig_barplot.update_xaxes(tickfont_size=10)
        
        return fig_barplot,scatter,mapfig,selected_df.to_json(date_format='iso', orient='split'),df2.to_json(date_format='iso', orient='split')
    else:
        return barplot_current_fig,scatter_current_fig,map_current_fig,features_current_data,features_current_data_general
@app.callback(
    Output('detail', 'figure'),
    Output('general', 'figure'),
    Input('barplot', 'clickData'),
    Input('features-detail-df', 'data'),
    Input('features-general-df', 'data'),
    Input('detail', 'figure'),
    Input('general', 'figure'),
    
    prevent_initial_call=True
)
def display_click_data(clickData,df_json, df_global_json, detail_current_fig, general_current_fig):
    if clickData is None:
        return detail_current_fig, general_current_fig
    
    detail_df = pd.read_json(df_json, orient='split')    
    geral_df = pd.read_json(df_global_json, orient='split')
    name = clickData['points'][0]['label']
    try:
        range_min = geral_df[name].min()
        range_max = geral_df[name].max()
    except:
        range_min = 100000
        range_max = -1
 
    #detail_df = detail_df.groupby(name).count().reset_index()
    #geral_df = geral_df.groupby(name).count().reset_index()
    
    if len(geral_df[name].unique().tolist()) > 12:
        #yy=[i for i in range(0, len(xx))]
        geral_xx = geral_df[name].tolist()
        detail_xx = detail_df[name].tolist()
        step = (range_max-range_min)/10
        fig = go.Figure([go.Histogram(x=detail_xx,xbins=dict(size=step))])
        fig.update_xaxes(range=[range_min-step, range_max+step])
        fig.update_layout(bargap=0.3)
        
        geral_fig = go.Figure([go.Histogram(x=geral_xx,xbins=dict(size=step))])
        geral_fig.update_xaxes(range=[range_min-step, range_max+step])
        geral_fig.update_layout(bargap=0.3)
    else: 
        geral_df = geral_df.groupby(name).count().reset_index()
        detail_df = detail_df.groupby(name).count().reset_index()
        
        geral_xx = geral_df[name].tolist()
        detail_xx = detail_df[name].tolist()
        
        geral_yy = geral_df[geral_df.columns[1]].tolist()
        detail_yy = detail_df[detail_df.columns[1]].tolist()
       
        yyy = []
        num = 0
        for i in geral_xx:
            if i in detail_xx:
                yyy.append(detail_yy[num])
                num +=1
            else:
                yyy.append(0)
        common_x = np.array(geral_xx).astype(str)
        fig = go.Figure([go.Bar(x=common_x, y=yyy)])
        geral_fig = go.Figure([go.Bar(x=common_x, y=geral_yy)])
    
    fig.update_layout(title_text=name,title_font_size=10,
            margin ={'l':0,'t':20,'b':0,'r':0},height=150)
    geral_fig.update_layout(title_text=name,title_font_size=10,
            margin ={'l':0,'t':20,'b':0,'r':0},height=150)
    return fig,geral_fig

if __name__ == "__main__":
   # app.run_server(debug=True,mode="inline")
    app.run_server()
    #df=df_backup

Dash app running on http://127.0.0.1:8050/
